In [1]:
#### 01 이미지 생성 #############################################################3
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import time
import os
import random
import math
import pyautogui
import pyperclip
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd

date = time.strftime("%Y%m%d")

# Excel 파일에서 키워드 불러오기
excel_file_path = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list_{date}.xlsx"
df = pd.read_excel(excel_file_path)
keywords = df['title'].dropna().tolist()

font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{date}/images"

# 버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 채도가 낮춰진 그라데이션 색상 조합
    soft_gradients = [
        [(220, 150, 150), (255, 255, 255)],  # 부드러운 분홍 → 흰색
        [(150, 180, 230), (255, 255, 255)],  # 부드러운 파랑 → 흰색
        [(230, 200, 150), (255, 255, 255)],  # 부드러운 주황 → 흰색
        [(170, 215, 170), (255, 255, 255)],  # 부드러운 연두 → 흰색
        [(190, 170, 220), (255, 255, 255)],  # 부드러운 보라 → 흰색
        [(220, 170, 200), (255, 255, 255)],  # 부드러운 핑크 → 흰색
        [(150, 190, 200), (255, 255, 255)],  # 부드러운 옥색 → 흰색
    ]
    
    # 랜덤한 그라데이션 색상 선택
    gradient_colors = random.choice(soft_gradients)
    start_color, end_color = gradient_colors
    
    # 빈 이미지 생성
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    
    # 랜덤 그라데이션 방향 선택
    direction = random.choice(['left_to_right', 'right_to_left', 'top_to_bottom', 'bottom_to_top', 'diagonal_tl', 'diagonal_tr'])
    
    # 그라데이션 생성 (다양한 방향)
    for y in range(height):
        for x in range(width):
            # 방향에 따른 비율 계산 (0.0 ~ 1.0)
            if direction == 'left_to_right':
                ratio = x / width
            elif direction == 'right_to_left':
                ratio = 1 - (x / width)
            elif direction == 'top_to_bottom':
                ratio = y / height
            elif direction == 'bottom_to_top':
                ratio = 1 - (y / height)
            elif direction == 'diagonal_tl':  # 왼쪽 상단에서 오른쪽 하단
                ratio = (x + y) / (width + height)
            else:  # diagonal_tr: 오른쪽 상단에서 왼쪽 하단
                ratio = (width - x + y) / (width + height)
            
            # 부드러운 변화를 위해 지수 조정 (1.3으로 낮춤)
            ratio = pow(ratio, 1.3)  # 더 부드러운 전환을 위해 지수 감소
            
            # 색상 보간
            r = int(start_color[0] * (1 - ratio) + end_color[0] * ratio)
            g = int(start_color[1] * (1 - ratio) + end_color[1] * ratio)
            b = int(start_color[2] * (1 - ratio) + end_color[2] * ratio)
            
            # 픽셀 색상 설정
            image.putpixel((x, y), (r, g, b))
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 더 진한 회색 색상 정의 (더 선명하게)
    dark_gray_color = (30, 30, 30)  # 더 진한 회색
    
    # 텍스트에 테두리 효과 추가 (더 선명하게 보이도록)
    # 텍스트 그림자 효과 (약간의 오프셋으로 그림자 추가)
    offset = 3
    shadow_color = (220, 220, 220)
    draw.multiline_text((text_x+offset, text_y+offset), formatted_text, font=font, fill=shadow_color, align="center", spacing=80)

    # 텍스트 그리기
    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 선명도 향상 (적당히)
    image = image.filter(ImageFilter.SHARPEN)
    
    # 이미지 파일명에 그라데이션 방향 정보 추가
    save_path = os.path.join(banner_save_folder, f"{filename}")
    image.save(save_path, quality=95, dpi=(300, 300))
    print(f"생성 완료: {save_path}")
    return save_path

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)

# 각 키워드에 대해 이미지 1개씩 생성
for keyword in keywords:
    # 키워드별 폴더 생성
    keyword_folder = os.path.join(image_final_folder, keyword)
    os.makedirs(keyword_folder, exist_ok=True)
    
    # 배너 이미지 생성 (각 키워드당 1개만)
    filename = f"banner_{keyword}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)

print("-------- 01이미지 생성 완료 ---------")
#### 01 이미지 생성 완료 #############################################################

#### 02 이미지 변환 ################################################################
from PIL import Image
import piexif
from piexif import helper
import datetime
import os
import xml.etree.ElementTree as ET
from io import BytesIO
import time

date = time.strftime("%Y%m%d")

# 이미지 폴더 경로 (하위 폴더 포함)
base_image_folder = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{date}/images"

# WebP 저장 폴더
webp_output_folder = os.path.join(base_image_folder, "webp")
os.makedirs(webp_output_folder, exist_ok=True)

# 지원하는 이미지 확장자
supported_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']

# 변환 결과 통계
total_files = 0
converted_files = 0
total_original_size = 0
total_jpeg_size = 0
total_webp_size = 0

# XMP 메타데이터 생성 함수
def create_xmp_metadata(keyword):
    # XMP 네임스페이스 정의
    xmp_ns = {
        'x': 'adobe:ns:meta/',
        'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
        'xmp': 'http://ns.adobe.com/xap/1.0/',
        'dc': 'http://purl.org/dc/elements/1.1/',
        'photoshop': 'http://ns.adobe.com/photoshop/1.0/',
    }
    
    # 현재 날짜/시간
    current_datetime = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    
    # XMP 루트 요소 생성
    root = ET.Element('{' + xmp_ns['x'] + '}xmpmeta')
    for prefix, uri in xmp_ns.items():
        root.set(f'xmlns:{prefix}', uri)
    
    # RDF 요소 추가
    rdf = ET.SubElement(root, '{' + xmp_ns['rdf'] + '}RDF')
    
    # Description 요소 추가
    desc = ET.SubElement(rdf, '{' + xmp_ns['rdf'] + '}Description')
    desc.set('{' + xmp_ns['dc'] + '}title', keyword)
    desc.set('{' + xmp_ns['dc'] + '}description', f"{keyword}의 공식 배너 이미지")
    desc.set('{' + xmp_ns['xmp'] + '}CreatorTool', f"{keyword}_웹사이트")
    desc.set('{' + xmp_ns['xmp'] + '}CreateDate', current_datetime)
    desc.set('{' + xmp_ns['photoshop'] + '}Headline', keyword)
    
    # XML을 문자열로 변환
    xmp_str = ET.tostring(root, encoding='utf-8')
    return xmp_str

# 모든 하위 폴더 순회하며 이미지 처리
for root_folder, directories, filenames in os.walk(base_image_folder):
    # webp 폴더는 제외
    if "webp" in root_folder:
        continue
    
    for filename in filenames:
        file_path = os.path.join(root_folder, filename)
        
        # 파일이 아니거나 지원하는 이미지 확장자가 아니면 건너뛰기
        if not os.path.isfile(file_path):
            continue
            
        file_ext = os.path.splitext(filename)[1].lower()
        if file_ext not in supported_extensions and file_ext != '.webp':
            continue
        
        total_files += 1
        
        try:
            # 파일 이름에서 키워드 생성 (확장자 제외)
            file_name = os.path.splitext(filename)[0]
            keyword = file_name.replace('-', ' ').replace('_', ' ')
            
            # 현재 날짜 시간
            current_datetime = datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")
            
            # 이미지 열기
            img = Image.open(file_path)
            
            # 원본 파일 크기
            original_size = os.path.getsize(file_path) / 1024  # KB로 변환
            total_original_size += original_size
            
            # 메타데이터 준비 (JPEG용)
            zeroth_ifd = {
                piexif.ImageIFD.Make: keyword.encode('utf-8'),
                piexif.ImageIFD.XResolution: (300, 1),
                piexif.ImageIFD.YResolution: (300, 1),
                piexif.ImageIFD.Software: f"{keyword}_웹사이트".encode('utf-8')
            }
            
            # UserComment 생성
            comment_text = f"{keyword}의 공식 배너 이미지"
            user_comment = helper.UserComment.dump(comment_text, "unicode")
            
            exif_ifd = {
                piexif.ExifIFD.DateTimeOriginal: current_datetime.encode('utf-8'),
                piexif.ExifIFD.LensMake: keyword.encode('utf-8'),
                piexif.ExifIFD.UserComment: user_comment
            }
            
            # EXIF 데이터 생성
            exif_dict = {"0th": zeroth_ifd, "Exif": exif_ifd}
            exif_bytes = piexif.dump(exif_dict)
            
            # WebP용 XMP 메타데이터 생성
            xmp_metadata = create_xmp_metadata(keyword)
            
            # 파일명 생성 (JPG 버전)
            new_filename = "-".join(file_name.split())
            jpeg_output_path = os.path.join(root_folder, new_filename + ".jpg")
            
            # JPEG으로 저장 (EXIF 메타데이터 포함)
            img.save(jpeg_output_path, format="JPEG", quality=85, exif=exif_bytes)
            
            # WebP 파일명 생성 (webp 폴더에 저장)
            webp_output_path = os.path.join(webp_output_folder, new_filename + ".webp")
            
            # WebP 형식으로 변환하여 저장 (XMP 메타데이터 포함)
            # WebP 라이브러리는 XMP 메타데이터를 직접 지원하지 않으므로 부가 정보로 저장
            try:
                img.save(webp_output_path, format="WebP", quality=85, method=6, 
                         exif=exif_bytes,  # 일부 WebP 구현에서는 EXIF를 부분적으로 지원할 수 있음
                         xmp=xmp_metadata)  # XMP 메타데이터 추가 (PIL 1.9.0 이상에서 지원)
            except TypeError:
                # 구버전 PIL에서 xmp 매개변수 지원하지 않을 수 있음
                img.save(webp_output_path, format="WebP", quality=85, method=6, 
                         exif=exif_bytes)
            
            # 파일 크기 계산
            jpeg_size = os.path.getsize(jpeg_output_path) / 1024  # KB로 변환
            webp_size = os.path.getsize(webp_output_path) / 1024  # KB로 변환
            
            total_jpeg_size += jpeg_size
            total_webp_size += webp_size
            
            print(f"이미지 처리 완료: {filename}")
            print(f"- 키워드: {keyword}")
            print(f"- 원본 파일 크기: {original_size:.2f} KB")
            print(f"- JPEG 파일 크기: {jpeg_size:.2f} KB")
            print(f"- WebP 파일 크기: {webp_size:.2f} KB")
            print(f"- JPEG 대비 WebP 절약: {jpeg_size - webp_size:.2f} KB ({(1 - webp_size/jpeg_size) * 100:.1f}% 감소)")
            print("-" * 50)
            
            converted_files += 1
            
        except Exception as e:
            print(f"오류 발생: {filename} - {str(e)}")

# 최종 결과 출력
print("\n======= 변환 완료 요약 =======")
print(f"총 이미지 파일: {total_files}")
print(f"변환 성공: {converted_files}")
print(f"변환 실패: {total_files - converted_files}")
print(f"총 원본 크기: {total_original_size:.2f} KB")
print(f"총 JPEG 크기: {total_jpeg_size:.2f} KB")
print(f"총 WebP 크기: {total_webp_size:.2f} KB")
print(f"총 절약된 용량(JPEG 대비): {total_jpeg_size - total_webp_size:.2f} KB ({(1 - total_webp_size/total_jpeg_size) * 100:.1f}% 감소)")
print(f"WebP 파일들은 {webp_output_folder}에 저장되었습니다.")
print("======= 이미지 변환 완료 =======")
#### 02 이미지 변환 완료 ###############################################################


#### 03 이미지 업로드 ###################################################################

import os
import base64
import requests
import json
from pathlib import Path
import mimetypes
import datetime
import pandas as pd

# WordPress 설정
WP_URL = "https://prayer-church.co.kr"
WP_API = f"{WP_URL}/wp-json/wp/v2"
USERNAME = "prayerchurch"
APP_PASSWORD = "AziD L62i Ga4b 1efW ToAs hBMB"
date = time.strftime("%Y%m%d")

FOLDER_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{date}/images/webp/"

REPORT_PATH = f"/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/{date}/report"

def get_image_files_recursive(folder_path):
    """지정된 폴더와 모든 하위 폴더에서 이미지 파일 가져오기"""
    이미지_확장자 = ['.jpg', '.jpeg', '.png', '.gif', '.webp']
    이미지_파일들 = []
    
    # os.walk를 사용하여 폴더와 모든 하위 폴더 탐색
    for 현재_경로, 하위_폴더들, 파일들 in os.walk(folder_path):
        for 파일 in 파일들:
            if any(파일.lower().endswith(확장자) for 확장자 in 이미지_확장자):
                이미지_파일들.append(os.path.join(현재_경로, 파일))
    
    return 이미지_파일들

def get_metadata_from_filename(filename):
    """파일명에서 메타데이터 추출"""
    # 경로와 확장자 없이 파일명만 가져오기
    기본_이름 = os.path.basename(filename)
    확장자_없는_이름 = os.path.splitext(기본_이름)[0]
    
    # 하이픈을 공백으로 대체
    이름_텍스트 = 확장자_없는_이름.replace('-', ' ')
    
    # 메타데이터 생성
    대체_텍스트 = 이름_텍스트
    캡션 = f"{이름_텍스트}"
    제목 = 이름_텍스트.title()  # 각 단어의 첫 글자 대문자화
    
    return {
        'alt_text': 대체_텍스트,
        'caption': 캡션,
        'title': 제목
    }

def create_schema_markup(metadata, image_url):
    """이미지용 schema.org 마크업 생성"""
    스키마 = {
        "@context": "https://schema.org/",
        "@type": "ImageObject",
        "contentUrl": image_url,
        "name": metadata['title'],
        "description": metadata['alt_text'],
        "caption": metadata['caption']
    }
    
    # ensure_ascii=False를 추가하여 한글이 올바르게 인코딩되도록 설정
    return json.dumps(스키마, ensure_ascii=False)

def upload_to_wordpress(image_path, metadata):
    """메타데이터와 함께 WordPress에 이미지 업로드"""
    try:
        # 인증 헤더 - Basic Auth
        인증 = requests.auth.HTTPBasicAuth(USERNAME, APP_PASSWORD)
        
        # 파일 이름 추출
        파일_이름 = os.path.basename(image_path)
        
        # 이미지 크기 정보 가져오기
        from PIL import Image
        with Image.open(image_path) as img:
            이미지_너비, 이미지_높이 = img.size
        
        # multipart/form-data로 파일 업로드
        with open(image_path, 'rb') as img:
            파일_데이터 = {'file': (파일_이름, img)}
            
            # 미디어 엔드포인트 POST 요청
            업로드_URL = f"{WP_API}/media"
            
            # 파일 업로드
            응답 = requests.post(
                업로드_URL,
                auth=인증,
                files=파일_데이터
            )
        
        if 응답.status_code in [200, 201]:
            응답_데이터 = 응답.json()
            이미지_URL = 응답_데이터.get('source_url', '')
            이미지_ID = 응답_데이터.get('id', '')
            
            # 업로드 성공 후 메타데이터 업데이트
            if 이미지_ID:
                # schema.org 마크업 생성 (이제 URL 있음)
                업데이트_스키마 = create_schema_markup(metadata, 이미지_URL)
                
                # 이미지 메타데이터 업데이트 - width와 height 추가
                업데이트_데이터 = {
                    'title': metadata['title'],
                    'alt_text': metadata['alt_text'],
                    'caption': metadata['caption'],
                    'description': 업데이트_스키마,
                    'meta': {
                        'width': 이미지_너비,
                        'height': 이미지_높이
                    }
                }
                
                # 메타데이터 업데이트 요청 (JSON 형식)
                헤더 = {
                    'Content-Type': 'application/json',
                    'Accept': 'application/json'
                }
                
                업데이트_URL = f"{WP_API}/media/{이미지_ID}"
                업데이트_응답 = requests.post(
                    업데이트_URL,
                    auth=인증,
                    headers=헤더,
                    json=업데이트_데이터
                )
                
                # 업데이트 결과 확인
                if 업데이트_응답.status_code not in [200, 201]:
                    print(f"메타데이터 업데이트 실패: {업데이트_응답.text}")
                
            return {
                'success': True,
                'id': 이미지_ID,
                'url': 이미지_URL,
                'width': 이미지_너비,
                'height': 이미지_높이,
                'title': metadata['title'],
                'alt_text': metadata['alt_text'],
                'caption': metadata['caption'],
                'response': 응답_데이터
            }
        else:
            return {
                'success': False,
                'status_code': 응답.status_code,
                'error': 응답.text
            }
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

def save_results_to_excel(results, folder_path):
    """결과를 엑셀 파일로 저장"""
    # 오늘 날짜 가져오기
    오늘 = datetime.datetime.now().strftime("%Y%m%d")
    
    # 결과 데이터 준비
    엑셀_데이터 = []
    
    for 항목 in results:
        파일명 = 항목['file_name']
        파일경로 = 항목['file_path']  # 전체 파일 경로 추가
        업로드_결과 = 항목['upload_result']
        
        if 업로드_결과['success']:
            행_데이터 = {
                '파일명': 파일명,
                '파일경로': 파일경로,  # 전체 파일 경로 추가
                '상태': '성공',
                'ID': 업로드_결과.get('id', ''),
                'URL': 업로드_결과.get('url', ''),
                '제목': 업로드_결과.get('title', ''),
                '대체텍스트': 업로드_결과.get('alt_text', ''),
                '캡션': 업로드_결과.get('caption', '')
            }
        else:
            행_데이터 = {
                '파일명': 파일명,
                '파일경로': 파일경로,  # 전체 파일 경로 추가
                '상태': '실패',
                '오류 코드': 업로드_결과.get('status_code', ''),
                '오류 메시지': 업로드_결과.get('error', '')
            }
        
        엑셀_데이터.append(행_데이터)
    
    # 데이터프레임 생성
    df = pd.DataFrame(엑셀_데이터)
    
    # 엑셀 파일 이름 생성
    파일명 = f"image_upload_report_{오늘}.xlsx"
    파일_경로 = os.path.join(folder_path, 파일명)
    
    # 지정된 폴더가 없으면 생성
    os.makedirs(folder_path, exist_ok=True)
    
    # 엑셀 파일로 저장
    df.to_excel(파일_경로, index=False, engine='openpyxl')
    
    return 파일_경로

def main():
    # 폴더와 모든 하위 폴더에서 이미지 파일 가져오기
    이미지_파일들 = get_image_files_recursive(FOLDER_PATH)
    
    # 이미지 개수 확인
    print("--- 이미지 업로드 시작----")
    print(f"총 {len(이미지_파일들)}개의 이미지 파일을 찾았습니다.")
    
    결과 = []
    
    # 각 이미지 처리
    for 인덱스, 이미지_경로 in enumerate(이미지_파일들, 1):
        파일_이름 = os.path.basename(이미지_경로)
        상대_경로 = os.path.relpath(이미지_경로, FOLDER_PATH)
        
        print(f"[{인덱스}/{len(이미지_파일들)}] '{상대_경로}' 처리 중...")
        
        # 파일명에서 메타데이터 추출
        메타데이터 = get_metadata_from_filename(이미지_경로)
        
        # WordPress에 업로드
        업로드_결과 = upload_to_wordpress(이미지_경로, 메타데이터)
        
        # 결과 저장
        이미지_정보 = {
            'file_name': 파일_이름,
            'file_path': 상대_경로,  # 상대 경로 저장
            'upload_result': 업로드_결과
        }
        결과.append(이미지_정보)
        
        # 결과 출력
        if 업로드_결과['success']:
            print(f"✅ 성공: '{상대_경로}' 업로드 완료, URL: {업로드_결과['url']}")
        else:
            print(f"❌ 실패: '{상대_경로}' 업로드 실패, 오류: {업로드_결과.get('error', '알 수 없는 오류')}")
    
    # 모든 결과 요약
    성공_수 = sum(1 for 항목 in 결과 if 항목['upload_result']['success'])
    실패_수 = len(결과) - 성공_수
    
    print("\n===== 업로드 결과 요약 =====")
    print(f"총 이미지: {len(결과)}")
    print(f"성공: {성공_수}")
    print(f"실패: {실패_수}")
    
    # 모든 성공한 URL 출력
    if 성공_수 > 0:
        print("\n===== 업로드된 이미지 URL =====")
        for 항목 in 결과:
            if 항목['upload_result']['success']:
                print(f"{항목['file_path']}: {항목['upload_result']['url']}")
    
    # 결과를 엑셀 파일로 저장
    엑셀_파일_경로 = save_results_to_excel(결과, REPORT_PATH)
    print(f"\n✅ 결과가 엑셀 파일로 저장되었습니다: {엑셀_파일_경로}")

if __name__ == "__main__":
    main()
    
print("----- 이미지 업로드 완료 -------")
#### 03 이미지 업로드 완료  #############################################################

print("############################## 모든 작업 완료 ##############################")



생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 10월/banner_새벽예배 대표기도문 10월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 11월/banner_새벽예배 대표기도문 11월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 12월/banner_새벽예배 대표기도문 12월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 6월 둘째주/banner_새벽예배 대표기도문 6월 둘째주.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 6월 셋째주/banner_새벽예배 대표기도문 6월 셋째주.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 6월 넷째주/banner_새벽예배 대표기도문 6월 넷째주.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 6월 마지막주/banner_새벽예배 대표기도문 6월 마지막주.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/새벽예배 대표기도문 7월 첫째주/banner_새벽예배 대표기도문 7월 첫째주.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723

생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/12월 셋째주 주일 낮예배 대표 기도문/banner_12월 셋째주 주일 낮예배 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/12월 넷째주 주일 낮예배 대표 기도문/banner_12월 넷째주 주일 낮예배 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 1월/banner_주일 낮예배 대표 기도문 1월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 2월/banner_주일 낮예배 대표 기도문 2월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 3월/banner_주일 낮예배 대표 기도문 3월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 4월/banner_주일 낮예배 대표 기도문 4월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 5월/banner_주일 낮예배 대표 기도문 5월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250723/images/주일 낮예배 대표 기도문 6월/banner_주일 낮예배 대표 기도문 6월.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsen

이미지 처리 완료: banner_새벽예배 대표기도문 8월 넷째주.png
- 키워드: banner 새벽예배 대표기도문 8월 넷째주
- 원본 파일 크기: 51.50 KB
- JPEG 파일 크기: 55.60 KB
- WebP 파일 크기: 16.30 KB
- JPEG 대비 WebP 절약: 39.30 KB (70.7% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 12월 넷째주.png
- 키워드: banner 새벽예배 대표기도문 12월 넷째주
- 원본 파일 크기: 47.16 KB
- JPEG 파일 크기: 53.29 KB
- WebP 파일 크기: 14.96 KB
- JPEG 대비 WebP 절약: 38.33 KB (71.9% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 6월 둘째주.png
- 키워드: banner 새벽예배 대표기도문 6월 둘째주
- 원본 파일 크기: 46.35 KB
- JPEG 파일 크기: 51.44 KB
- WebP 파일 크기: 15.52 KB
- JPEG 대비 WebP 절약: 35.92 KB (69.8% 감소)
--------------------------------------------------
이미지 처리 완료: banner_주일 낮예배 대표 기도문 1월.png
- 키워드: banner 주일 낮예배 대표 기도문 1월
- 원본 파일 크기: 48.28 KB
- JPEG 파일 크기: 50.47 KB
- WebP 파일 크기: 15.15 KB
- JPEG 대비 WebP 절약: 35.32 KB (70.0% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 12월 첫째주.png
- 키워드: banner 새벽예배 대표기도문 12월 첫째주
- 원본 파일 크기

이미지 처리 완료: banner_새벽예배 대표기도문 12월 셋째주.png
- 키워드: banner 새벽예배 대표기도문 12월 셋째주
- 원본 파일 크기: 49.06 KB
- JPEG 파일 크기: 56.52 KB
- WebP 파일 크기: 17.06 KB
- JPEG 대비 WebP 절약: 39.47 KB (69.8% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 10월 둘째주.png
- 키워드: banner 새벽예배 대표기도문 10월 둘째주
- 원본 파일 크기: 43.67 KB
- JPEG 파일 크기: 49.80 KB
- WebP 파일 크기: 15.25 KB
- JPEG 대비 WebP 절약: 34.55 KB (69.4% 감소)
--------------------------------------------------
이미지 처리 완료: banner_주일 낮예배 대표 기도문 9월.png
- 키워드: banner 주일 낮예배 대표 기도문 9월
- 원본 파일 크기: 56.09 KB
- JPEG 파일 크기: 55.83 KB
- WebP 파일 크기: 18.19 KB
- JPEG 대비 WebP 절약: 37.64 KB (67.4% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 9월 첫째주.png
- 키워드: banner 새벽예배 대표기도문 9월 첫째주
- 원본 파일 크기: 54.27 KB
- JPEG 파일 크기: 57.58 KB
- WebP 파일 크기: 18.30 KB
- JPEG 대비 WebP 절약: 39.28 KB (68.2% 감소)
--------------------------------------------------
이미지 처리 완료: banner_새벽예배 대표기도문 12월 마지막주.png
- 키워드: banner 새벽예배 대표기도문 12월 마지막주
- 원본 파

✅ 성공: 'banner_6월-넷째주-주일-낮예배-대표-기도문.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/07/banner_6월-넷째주-주일-낮예배-대표-기도문.webp
[33/75] 'banner_7월-첫째주-주일-낮예배-대표-기도문.webp' 처리 중...
✅ 성공: 'banner_7월-첫째주-주일-낮예배-대표-기도문.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/07/banner_7월-첫째주-주일-낮예배-대표-기도문.webp
[34/75] 'banner_10월-둘째주-주일-낮예배-대표-기도문.webp' 처리 중...
✅ 성공: 'banner_10월-둘째주-주일-낮예배-대표-기도문.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/07/banner_10월-둘째주-주일-낮예배-대표-기도문.webp
[35/75] 'banner_새벽예배-대표기도문-8월-마지막주.webp' 처리 중...
✅ 성공: 'banner_새벽예배-대표기도문-8월-마지막주.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/07/banner_새벽예배-대표기도문-8월-마지막주.webp
[36/75] 'banner_9월-셋째주-주일-낮예배-대표-기도문.webp' 처리 중...
✅ 성공: 'banner_9월-셋째주-주일-낮예배-대표-기도문.webp' 업로드 완료, URL: https://prayer-church.co.kr/wp-content/uploads/2025/07/banner_9월-셋째주-주일-낮예배-대표-기도문.webp
[37/75] 'banner_새벽예배-대표기도문-12월-둘째주.webp' 처리 중...
✅ 성공: 'banner_새벽예배-대표기도문-12월-둘째주.webp' 업